In [1]:
import numpy as np
import math

In [2]:
UNCLASSIFIED = False
NOISE = None

In [4]:
#计算距离
def _dist(p,q):
    return math.sqrt(np.power(p-q,2).sum())

In [5]:
def _eps_neighborhood(p,q,eps):
    return _dist(p,q)< eps

In [6]:
def _region_query(m,point_id,eps):
    n_points = m.shape[1]
    seeds = []
    for i in range(n_points):
        if _eps_neighborhood(m[:,i],m[:,point_id],eps):
            seeds.append(i)
    return seeds

In [20]:
def _expand_cluster(m,classifications,point_id,cluster_id,eps,min_points):
    seeds = _region_query(m,point_id,eps)
    if (len(seeds)) < min_points:
        classifications[point_id] = NOISE
        return False
    else:
        classifications[point_id] = cluster_id
        for seed_id in seeds:
            classifications[seed_id] = cluster_id
        while(len(seeds) > 0):
            current_point = seeds[0]
            results = _region_query(m,current_point,eps)
            if len(results)>= min_points:
                for i in range(0,len(results)):
                    result_point = results[i]
                    if (classifications[result_point] ==UNCLASSIFIED) or (classifications[result_point]) == NOISE:
                        if classifications[result_point] == UNCLASSIFIED:
                            seeds.append(result_point)
                        classifications[result_point] = cluster_id
            seeds = seeds[1:]
        return True
                        

In [23]:
def dbscan(m,eps,min_point):
    cluster_id = 1
    n_points = m.shape[1]
    classifications = [UNCLASSIFIED]*n_points
    for point_id in range(0,n_points):
        point = m[:,point_id]
        if classifications[point_id] == UNCLASSIFIED:
            if _expand_cluster(m,classifications,point_id,cluster_id,eps,min_point):
                cluster_id =cluster_id+1
    return classifications

In [24]:
def test_dbscan():
    m = np.matrix('1 1.2 0.8 3.7 3.9 3.6 10; 1.1 0.8 1 4 3.9 4.1 10')
    eps = 0.5
    min_points = 2
    #return dbscan(m, eps, min_points)
    assert dbscan(m, eps, min_points) == [1, 1, 1, 2, 2, 2, None]

In [26]:
test_dbscan()

[1, 1, 1, 2, 2, 2, None]